# **Data Import**

In [1]:
import pandas as pd
import numpy as np

Looking at the zipfile

In [ ]:
import zipfile
# python module used to open, read and extraxt from zipfiles

with zipfile.ZipFile('t20s_male_csv2.zip', 'r') as all_it20s:
# turn the zipfile into an object so we can do some stuff wirh it
  it20s_namelist = all_it20s.namelist()
  print(it20s_namelist[:10])



['README.txt', '211048.csv', '211048_info.csv', '211028.csv', '211028_info.csv', '222678.csv', '222678_info.csv', '226374.csv', '226374_info.csv', '237242.csv']


In [ ]:
len(it20s_namelist)

5241

In [ ]:
with zipfile.ZipFile('t20s_male_csv2.zip', 'r') as all_it20s: # have to use zoipfile each time to access the zip
    with all_it20s.open('211048_info.csv') as f:
        lines = f.readlines()
        for i, line in enumerate(lines[:10], 1):
            print(f"Line {i}: {line.decode('utf-8').strip()}")
# utf-8 decode turn it from byte string to normal string, as we have different number of columns in some of the rows

Line 1: version,2.1.0
Line 2: info,balls_per_over,6
Line 3: info,team,New Zealand
Line 4: info,team,Australia
Line 5: info,gender,male
Line 6: info,season,2004/05
Line 7: info,date,2005/02/17
Line 8: info,venue,Eden Park
Line 9: info,city,Auckland
Line 10: info,toss_winner,Australia


Want to only include games between Eng and Aus

The info files contain what teams are playing

In [ ]:
it20s_info_files = [f for f in it20s_namelist if f.endswith('_info.csv')]
# using [] to create a list of every file in the filelist that ends in _info.csv
print(len(it20s_info_files))

2620


Half the files have _info at the end which is what we want. Also -1 for the README file

In [ ]:
eng_vs_aus_matches = []

with zipfile.ZipFile('t20s_male_csv2.zip', 'r') as all_it20s:
    for f in it20s_info_files:
        with all_it20s.open(f) as file:
            lines = file.readlines()
            line3 = lines[2].decode('utf-8')
            line4 = lines[3].decode('utf-8')
            # lines 2 and 3 actually 3 and 4 with 0 index
            # decode each line from byte to string

            if "England" in line3 and "Australia" in line4 or \
               "Australia" in line3 and "England" in line4:# check both orders aus/eng and eng/aus
                eng_vs_aus_matches.append(f)
# gives list with info docs that are between eng and aus

In [ ]:
print(len(eng_vs_aus_matches))

26


In [ ]:
match_ids = [f.replace('_info.csv', '') for f in eng_vs_aus_matches]
print(match_ids)
# gives me all the match ids for t20Is between E and A, now need to make a file with these in

['211028', '249227', '287861', '350050', '412703', '446960', '446961', '566937', '566938', '636164', '636165', '636166', '743975', '1072317', '1072318', '1119542', '1198235', '1198236', '1198237', '1273737', '1317486', '1317487', '1317488', '1415717', '1385697', '1385698']


In [ ]:
files_to_include = []
for mid in match_ids:
  files_to_include.append(f"{mid}.csv")
  files_to_include.append(f"{mid}_info.csv")

#use f string to insert

In [ ]:
source_zip = 't20s_male_csv2.zip'
output_zip = 'eng_aus_matches_only.zip'

with zipfile.ZipFile(source_zip, 'r') as zin:
    with zipfile.ZipFile(output_zip, 'w') as zout:
        for f in files_to_include:
            data = zin.read(f)
            zout.writestr(f, data)


In [ ]:
with zipfile.ZipFile("eng_aus_matches_only.zip", "r") as zip_ref:
    zip_ref.printdir()
# list the files in the zip

File Name                                             Modified             Size
211028.csv                                     2025-08-01 06:37:28        23981
211028_info.csv                                2025-08-01 06:37:28         2161
249227.csv                                     2025-08-01 06:37:28        29907
249227_info.csv                                2025-08-01 06:37:28         2232
287861.csv                                     2025-08-01 06:37:28        22655
287861_info.csv                                2025-08-01 06:37:28         2268
350050.csv                                     2025-08-01 06:37:28        14547
350050_info.csv                                2025-08-01 06:37:28         2223
412703.csv                                     2025-08-01 06:37:28        28866
412703_info.csv                                2025-08-01 06:37:28         2273
446960.csv                                     2025-08-01 06:37:28        26808
446960_info.csv                         

# **Intial Feature Engineering**


1.   These features arer going to be very basic and got from the existing CSV
2.   The initial df for one match id




When setting up ypu dataframe to be suitable for the TFT model I need a

1.   Group ID, to identify each independent sequence = .ID
2.   Time index, to provide temporal structure to each independent sequence = .TIME
3.   Features: These can be Static variables, past variables, future known variables
4.   Target Variable coulumns, what I am trying to predict  .TARGET










In [ ]:
with zipfile.ZipFile("eng_aus_matches_only.zip", "r") as zip_ref:
    with zip_ref.open("211028.csv") as file:
        df_2011028 = pd.read_csv(file)
# taking one eng aus game and converting the ball by ball csv into a pd df

In [ ]:
df_2011028.head(5)

,match_id,season,start_date,venue,innings,ball,batting_team,bowling_team,striker,non_striker,...,extras,wides,noballs,byes,legbyes,penalty,wicket_type,player_dismissed,other_wicket_type,other_player_dismissed
0,211028,2005,2005-06-13,The Rose Bowl,1,0.1,England,Australia,ME Trescothick,GO Jones,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,211028,2005,2005-06-13,The Rose Bowl,1,0.2,England,Australia,ME Trescothick,GO Jones,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,211028,2005,2005-06-13,The Rose Bowl,1,0.3,England,Australia,GO Jones,ME Trescothick,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,211028,2005,2005-06-13,The Rose Bowl,1,0.4,England,Australia,GO Jones,ME Trescothick,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,211028,2005,2005-06-13,The Rose Bowl,1,0.5,England,Australia,GO Jones,ME Trescothick,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Add **group ID** for TFT model

In [ ]:
# add match id and innings column as TFT expects a group id to identify independent sequences, each innings is different even if they are same match
df_2011028['match_id_innings'] = df_2011028['match_id'].astype(str)+'_'+df_2011028['innings'].astype(str)

df_2011028.head(1)

,match_id,season,start_date,venue,innings,ball,batting_team,bowling_team,striker,non_striker,...,wides,noballs,byes,legbyes,penalty,wicket_type,player_dismissed,other_wicket_type,other_player_dismissed,match_id_innings
0,211028,2005,2005-06-13,The Rose Bowl,1,0.1,England,Australia,ME Trescothick,GO Jones,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,211028_1


Add **time index** for TFT model

In [ ]:
df_2011028['time_idx'] = df_2011028.groupby('match_id_innings').cumcount() + 1

df_2011028.head(10)

,match_id,season,start_date,venue,innings,ball,batting_team,bowling_team,striker,non_striker,...,noballs,byes,legbyes,penalty,wicket_type,player_dismissed,other_wicket_type,other_player_dismissed,match_id_innings,time_idx
0,211028,2005,2005-06-13,The Rose Bowl,1,0.1,England,Australia,ME Trescothick,GO Jones,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,211028_1,1
1,211028,2005,2005-06-13,The Rose Bowl,1,0.2,England,Australia,ME Trescothick,GO Jones,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,211028_1,2
2,211028,2005,2005-06-13,The Rose Bowl,1,0.3,England,Australia,GO Jones,ME Trescothick,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,211028_1,3
3,211028,2005,2005-06-13,The Rose Bowl,1,0.4,England,Australia,GO Jones,ME Trescothick,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,211028_1,4
4,211028,2005,2005-06-13,The Rose Bowl,1,0.5,England,Australia,GO Jones,ME Trescothick,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,211028_1,5
5,211028,2005,2005-06-13,The Rose Bowl,1,0.6,England,Australia,GO Jones,ME Trescothick,...,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,211028_1,6
6,211028,2005,2005-06-13,The Rose Bowl,1,0.7,England,Australia,GO Jones,ME Trescothick,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,211028_1,7
7,211028,2005,2005-06-13,The Rose Bowl,1,1.1,England,Australia,ME Trescothick,GO Jones,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,211028_1,8
8,211028,2005,2005-06-13,The Rose Bowl,1,1.2,England,Australia,ME Trescothick,GO Jones,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,211028_1,9
9,211028,2005,2005-06-13,The Rose Bowl,1,1.3,England,Australia,ME Trescothick,GO Jones,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,211028_1,10


Add total_runs as a simple Target Variable

In [ ]:
df_2011028['total_runs_off_ball'] = df_2011028['runs_off_bat'].astype(int) + df_2011028['extras'].astype(int)
df_2011028.head(5)

,match_id,season,start_date,venue,innings,ball,batting_team,bowling_team,striker,non_striker,...,byes,legbyes,penalty,wicket_type,player_dismissed,other_wicket_type,other_player_dismissed,match_id_innings,time_idx,total_runs_off_ball
0,211028,2005,2005-06-13,The Rose Bowl,1,0.1,England,Australia,ME Trescothick,GO Jones,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,211028_1,1,0
1,211028,2005,2005-06-13,The Rose Bowl,1,0.2,England,Australia,ME Trescothick,GO Jones,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,211028_1,2,1
2,211028,2005,2005-06-13,The Rose Bowl,1,0.3,England,Australia,GO Jones,ME Trescothick,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,211028_1,3,0
3,211028,2005,2005-06-13,The Rose Bowl,1,0.4,England,Australia,GO Jones,ME Trescothick,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,211028_1,4,0
4,211028,2005,2005-06-13,The Rose Bowl,1,0.5,England,Australia,GO Jones,ME Trescothick,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,211028_1,5,0


Add cumulative runs

In [ ]:
df_2011028['cum_runs'] = df_2011028.groupby('match_id_innings')['total_runs_off_ball'].cumsum()
df_2011028.head(15)

,match_id,season,start_date,venue,innings,ball,batting_team,bowling_team,striker,non_striker,...,legbyes,penalty,wicket_type,player_dismissed,other_wicket_type,other_player_dismissed,match_id_innings,time_idx,total_runs_off_ball,cum_runs
0,211028,2005,2005-06-13,The Rose Bowl,1,0.1,England,Australia,ME Trescothick,GO Jones,...,NaN,NaN,NaN,NaN,NaN,NaN,211028_1,1,0,0
1,211028,2005,2005-06-13,The Rose Bowl,1,0.2,England,Australia,ME Trescothick,GO Jones,...,NaN,NaN,NaN,NaN,NaN,NaN,211028_1,2,1,1
2,211028,2005,2005-06-13,The Rose Bowl,1,0.3,England,Australia,GO Jones,ME Trescothick,...,NaN,NaN,NaN,NaN,NaN,NaN,211028_1,3,0,1
3,211028,2005,2005-06-13,The Rose Bowl,1,0.4,England,Australia,GO Jones,ME Trescothick,...,NaN,NaN,NaN,NaN,NaN,NaN,211028_1,4,0,1
4,211028,2005,2005-06-13,The Rose Bowl,1,0.5,England,Australia,GO Jones,ME Trescothick,...,NaN,NaN,NaN,NaN,NaN,NaN,211028_1,5,0,1
5,211028,2005,2005-06-13,The Rose Bowl,1,0.6,England,Australia,GO Jones,ME Trescothick,...,NaN,NaN,NaN,NaN,NaN,NaN,211028_1,6,1,2
6,211028,2005,2005-06-13,The Rose Bowl,1,0.7,England,Australia,GO Jones,ME Trescothick,...,NaN,NaN,NaN,NaN,NaN,NaN,211028_1,7,2,4
7,211028,2005,2005-06-13,The Rose Bowl,1,1.1,England,Australia,ME Trescothick,GO Jones,...,NaN,NaN,NaN,NaN,NaN,NaN,211028_1,8,0,4
8,211028,2005,2005-06-13,The Rose Bowl,1,1.2,England,Australia,ME Trescothick,GO Jones,...,NaN,NaN,NaN,NaN,NaN,NaN,211028_1,9,0,4
9,211028,2005,2005-06-13,The Rose Bowl,1,1.3,England,Australia,ME Trescothick,GO Jones,...,NaN,NaN,NaN,NaN,NaN,NaN,211028_1,10,1,5


Add cumulative wickets

In [ ]:
df_2011028['is_wicket'] = df_2011028['wicket_type'].notna().astype(int)
# not na, check if there is a nan value or something else, then as type turns to integer rather than t/f
df_2011028['cum_wickets'] = df_2011028.groupby('match_id_innings')['is_wicket'].cumsum()
# can now count number of wickets
df_2011028.head(100)

,match_id,season,start_date,venue,innings,ball,batting_team,bowling_team,striker,non_striker,...,wicket_type,player_dismissed,other_wicket_type,other_player_dismissed,match_id_innings,time_idx,total_runs_off_ball,cum_runs,is_wicket,cum_wickets
0,211028,2005,2005-06-13,The Rose Bowl,1,0.1,England,Australia,ME Trescothick,GO Jones,...,NaN,NaN,NaN,NaN,211028_1,1,0,0,0,0
1,211028,2005,2005-06-13,The Rose Bowl,1,0.2,England,Australia,ME Trescothick,GO Jones,...,NaN,NaN,NaN,NaN,211028_1,2,1,1,0,0
2,211028,2005,2005-06-13,The Rose Bowl,1,0.3,England,Australia,GO Jones,ME Trescothick,...,NaN,NaN,NaN,NaN,211028_1,3,0,1,0,0
3,211028,2005,2005-06-13,The Rose Bowl,1,0.4,England,Australia,GO Jones,ME Trescothick,...,NaN,NaN,NaN,NaN,211028_1,4,0,1,0,0
4,211028,2005,2005-06-13,The Rose Bowl,1,0.5,England,Australia,GO Jones,ME Trescothick,...,NaN,NaN,NaN,NaN,211028_1,5,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,211028,2005,2005-06-13,The Rose Bowl,1,15.2,England,Australia,PD Collingwood,AJ Strauss,...,NaN,NaN,NaN,NaN,211028_1,96,1,126,0,5
96,211028,2005,2005-06-13,The Rose Bowl,1,15.3,England,Australia,AJ Strauss,PD Collingwood,...,NaN,NaN,NaN,NaN,211028_1,97,0,126,0,5
97,211028,2005,2005-06-13,The Rose Bowl,1,15.4,England,Australia,AJ Strauss,PD Collingwood,...,NaN,NaN,NaN,NaN,211028_1,98,1,127,0,5
98,211028,2005,2005-06-13,The Rose Bowl,1,15.5,England,Australia,PD Collingwood,AJ Strauss,...,NaN,NaN,NaN,NaN,211028_1,99,4,131,0,5


Add lagged feature that shows runs from the previous 3 balls

In [ ]:
df_2011028['runs_prev_3']=df_2011028.groupby('match_id')['total_runs_off_ball'].shift(3)
# use shift to fidn the total of the last 3 balls
df_2011028['runs_prev_3'] = df_2011028['runs_prev_3'].fillna(0).astype(int)
# was originally returning a float as there was nan values for the first 3 balls. However, I have set it to zero. This is because this feature is to give an idea of momentum and at start of the innings there is little momentum so zero will have low impact. Also as we have time_idx the model woll recognise these balls as being early in the innings.
df_2011028.head(15)

,match_id,season,start_date,venue,innings,ball,batting_team,bowling_team,striker,non_striker,...,player_dismissed,other_wicket_type,other_player_dismissed,match_id_innings,time_idx,total_runs_off_ball,cum_runs,is_wicket,cum_wickets,runs_prev_3
0,211028,2005,2005-06-13,The Rose Bowl,1,0.1,England,Australia,ME Trescothick,GO Jones,...,NaN,NaN,NaN,211028_1,1,0,0,0,0,0
1,211028,2005,2005-06-13,The Rose Bowl,1,0.2,England,Australia,ME Trescothick,GO Jones,...,NaN,NaN,NaN,211028_1,2,1,1,0,0,0
2,211028,2005,2005-06-13,The Rose Bowl,1,0.3,England,Australia,GO Jones,ME Trescothick,...,NaN,NaN,NaN,211028_1,3,0,1,0,0,0
3,211028,2005,2005-06-13,The Rose Bowl,1,0.4,England,Australia,GO Jones,ME Trescothick,...,NaN,NaN,NaN,211028_1,4,0,1,0,0,0
4,211028,2005,2005-06-13,The Rose Bowl,1,0.5,England,Australia,GO Jones,ME Trescothick,...,NaN,NaN,NaN,211028_1,5,0,1,0,0,1
5,211028,2005,2005-06-13,The Rose Bowl,1,0.6,England,Australia,GO Jones,ME Trescothick,...,NaN,NaN,NaN,211028_1,6,1,2,0,0,0
6,211028,2005,2005-06-13,The Rose Bowl,1,0.7,England,Australia,GO Jones,ME Trescothick,...,NaN,NaN,NaN,211028_1,7,2,4,0,0,0
7,211028,2005,2005-06-13,The Rose Bowl,1,1.1,England,Australia,ME Trescothick,GO Jones,...,NaN,NaN,NaN,211028_1,8,0,4,0,0,0
8,211028,2005,2005-06-13,The Rose Bowl,1,1.2,England,Australia,ME Trescothick,GO Jones,...,NaN,NaN,NaN,211028_1,9,0,4,0,0,1
9,211028,2005,2005-06-13,The Rose Bowl,1,1.3,England,Australia,ME Trescothick,GO Jones,...,NaN,NaN,NaN,211028_1,10,1,5,0,0,2


Create over number so we can do powerplay and death overs

In [ ]:
df_2011028['over_number'] = df_2011028['ball'].apply(lambda x: int(x) + 1)
df_2011028.head(146)
# look at 146 as there was an over with a noball and was checking that it work even if there was more than 6 balls in the over and it did.

,match_id,season,start_date,venue,innings,ball,batting_team,bowling_team,striker,non_striker,...,other_wicket_type,other_player_dismissed,match_id_innings,time_idx,total_runs_off_ball,cum_runs,is_wicket,cum_wickets,runs_prev_3,over_number
0,211028,2005,2005-06-13,The Rose Bowl,1,0.1,England,Australia,ME Trescothick,GO Jones,...,NaN,NaN,211028_1,1,0,0,0,0,0,1
1,211028,2005,2005-06-13,The Rose Bowl,1,0.2,England,Australia,ME Trescothick,GO Jones,...,NaN,NaN,211028_1,2,1,1,0,0,0,1
2,211028,2005,2005-06-13,The Rose Bowl,1,0.3,England,Australia,GO Jones,ME Trescothick,...,NaN,NaN,211028_1,3,0,1,0,0,0,1
3,211028,2005,2005-06-13,The Rose Bowl,1,0.4,England,Australia,GO Jones,ME Trescothick,...,NaN,NaN,211028_1,4,0,1,0,0,0,1
4,211028,2005,2005-06-13,The Rose Bowl,1,0.5,England,Australia,GO Jones,ME Trescothick,...,NaN,NaN,211028_1,5,0,1,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
141,211028,2005,2005-06-13,The Rose Bowl,2,2.5,Australia,England,AC Gilchrist,ML Hayden,...,NaN,NaN,211028_2,17,0,23,1,1,1,3
142,211028,2005,2005-06-13,The Rose Bowl,2,2.6,Australia,England,ML Hayden,A Symonds,...,NaN,NaN,211028_2,18,0,23,1,2,1,3
143,211028,2005,2005-06-13,The Rose Bowl,2,2.7,Australia,England,A Symonds,MJ Clarke,...,NaN,NaN,211028_2,19,0,23,0,2,2,3
144,211028,2005,2005-06-13,The Rose Bowl,2,3.1,Australia,England,MJ Clarke,A Symonds,...,NaN,NaN,211028_2,20,0,23,1,3,0,4


In [ ]:
#powerplayer <= 6 and death overs >= 17
df_2011028['powerplay']=(df_2011028['over_number']<=6).astype(int)
df_2011028['death_overs']=(df_2011028['over_number']>=17).astype(int)
df_2011028.head(120)

# during the powerplay teams are only allowed 2 fielders outside of the 30 yard circle
# for the remaining overs you are not allowed more than 5 fielders outside the ring
# the last 4 overs are considered the death overs and bowling style is known to change, bowlers tend to try variations i.e slower balls to prevent batsmen scoring

,match_id,season,start_date,venue,innings,ball,batting_team,bowling_team,striker,non_striker,...,match_id_innings,time_idx,total_runs_off_ball,cum_runs,is_wicket,cum_wickets,runs_prev_3,over_number,powerplay,death_overs
0,211028,2005,2005-06-13,The Rose Bowl,1,0.1,England,Australia,ME Trescothick,GO Jones,...,211028_1,1,0,0,0,0,0,1,1,0
1,211028,2005,2005-06-13,The Rose Bowl,1,0.2,England,Australia,ME Trescothick,GO Jones,...,211028_1,2,1,1,0,0,0,1,1,0
2,211028,2005,2005-06-13,The Rose Bowl,1,0.3,England,Australia,GO Jones,ME Trescothick,...,211028_1,3,0,1,0,0,0,1,1,0
3,211028,2005,2005-06-13,The Rose Bowl,1,0.4,England,Australia,GO Jones,ME Trescothick,...,211028_1,4,0,1,0,0,0,1,1,0
4,211028,2005,2005-06-13,The Rose Bowl,1,0.5,England,Australia,GO Jones,ME Trescothick,...,211028_1,5,0,1,0,0,1,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,211028,2005,2005-06-13,The Rose Bowl,1,18.3,England,Australia,VS Solanki,PD Collingwood,...,211028_1,116,1,159,0,6,1,19,0,1
116,211028,2005,2005-06-13,The Rose Bowl,1,18.4,England,Australia,PD Collingwood,VS Solanki,...,211028_1,117,0,159,0,6,1,19,0,1
117,211028,2005,2005-06-13,The Rose Bowl,1,18.5,England,Australia,PD Collingwood,VS Solanki,...,211028_1,118,4,163,0,6,0,19,0,1
118,211028,2005,2005-06-13,The Rose Bowl,1,18.6,England,Australia,PD Collingwood,VS Solanki,...,211028_1,119,4,167,0,6,1,19,0,1


Add some static features from the info file
1. doing this basically now, will need to make a mother file with all info and link stuff we want to match_id

In [ ]:
info_dict_2011028 = {}

with zipfile.ZipFile("eng_aus_matches_only.zip", "r") as zip_ref:
    with zip_ref.open("211028_info.csv") as f:
        for line in f:
            parts = line.decode('utf-8').strip().split(",")
            if len(parts) == 3 and parts[0] == "info":
                key = parts[1].strip()
                value = parts[2].strip()
                info_dict_2011028[key] = value

# create empty dict and then we can add key value pairs with the information I want

In [ ]:
#set what we want
for key in ['city', 'toss_winner', 'toss_decision']:
  df_2011028[key] = info_dict_2011028[key]
  df_2011028.head(5)

In [ ]:
pd.set_option('display.max_columns', None)
df_2011028.head()


,match_id,season,start_date,venue,innings,ball,batting_team,bowling_team,striker,non_striker,bowler,runs_off_bat,extras,wides,noballs,byes,legbyes,penalty,wicket_type,player_dismissed,other_wicket_type,other_player_dismissed,match_id_innings,time_idx,total_runs_off_ball,cum_runs,is_wicket,cum_wickets,runs_prev_3,over_number,powerplay,death_overs,city,toss_winner,toss_decision
0,211028,2005,2005-06-13,The Rose Bowl,1,0.1,England,Australia,ME Trescothick,GO Jones,B Lee,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,211028_1,1,0,0,0,0,0,1,1,0,Southampton,England,bat
1,211028,2005,2005-06-13,The Rose Bowl,1,0.2,England,Australia,ME Trescothick,GO Jones,B Lee,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,211028_1,2,1,1,0,0,0,1,1,0,Southampton,England,bat
2,211028,2005,2005-06-13,The Rose Bowl,1,0.3,England,Australia,GO Jones,ME Trescothick,B Lee,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,211028_1,3,0,1,0,0,0,1,1,0,Southampton,England,bat
3,211028,2005,2005-06-13,The Rose Bowl,1,0.4,England,Australia,GO Jones,ME Trescothick,B Lee,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,211028_1,4,0,1,0,0,0,1,1,0,Southampton,England,bat
4,211028,2005,2005-06-13,The Rose Bowl,1,0.5,England,Australia,GO Jones,ME Trescothick,B Lee,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,211028_1,5,0,1,0,0,1,1,1,0,Southampton,England,bat


# **Eng/Aus Pipeline**

### Input features

Now add all those features I've added to a dataframe for all the eng and aus matches

In [ ]:
# function tp add all the features i have already created
def process_match(ball_df, info_dict):
    df = ball_df.copy()

    # match_id_innings
    df['match_id_innings'] = df['match_id'].astype(str) + "_" + df['innings'].astype(str)
    # time_idx
    df['time_idx'] = df.groupby('match_id_innings').cumcount() + 1
    # Total runs off ball
    df['total_runs_off_ball'] = df['runs_off_bat'] + df['extras']
    # Cumulative runs
    df['cum_runs'] = df.groupby('match_id_innings')['total_runs_off_ball'].cumsum()
    # wickets
    df['is_wicket'] = df['wicket_type'].notna().astype(int)
    df['cum_wickets'] = df.groupby('match_id_innings')['is_wicket'].cumsum()
    # previous 3 balls
    df['runs_prev_3'] = df.groupby('match_id_innings')['total_runs_off_ball'].shift(3).fillna(0).astype(int)
    # Over number
    df['over_number'] = df['ball'].apply(lambda x: int(x) + 1)
    # Powerplay and death overs
    df['powerplay'] = (df['over_number'] <= 6).astype(int)
    df['death_overs'] = (df['over_number'] >= 17).astype(int)

    # Balls remaing
    #legal delivery column, cum count it then subtract from 120
    df['legal_delivery'] = ((df['wides'].fillna(0) == 0) & (df['noballs'].fillna(0) == 0)).astype(int)
    df['legal_delivery_number'] = df.groupby('match_id_innings')['legal_delivery'].cumsum()
    df['balls_remaining'] = 120 - df['legal_delivery_number']


    # Static covariates from info file
    for key in ['city', 'toss_winner', 'toss_decision']:
        df[key] = info_dict.get(key, None)


    return df



In [ ]:
zip_path = "eng_aus_matches_only.zip"

all_eng_aus_matches = []

# Open the zip file
with zipfile.ZipFile(zip_path, "r") as zip_ref:
    for match_id in match_ids:
        ball_file = f"{match_id}.csv"
        info_file = f"{match_id}_info.csv"

        with zip_ref.open(ball_file) as f:
            ball_df = pd.read_csv(f)
            # look at the ball by ball file

        info_dict = {}
        with zip_ref.open(info_file) as f:
            for line in f:
                parts = line.decode('utf-8').strip().split(",")
                if len(parts) == 3 and parts[0] == "info":
                    key = parts[1].strip()
                    value = parts[2].strip()
                    info_dict[key] = value
                    # parse the info file for static covariates

        processed_df = process_match(ball_df, info_dict)
        all_eng_aus_matches.append(processed_df)

df_all_eng_aus = pd.concat(all_eng_aus_matches, ignore_index=True)

df_all_eng_aus.head(127)


,match_id,season,start_date,venue,innings,ball,batting_team,bowling_team,striker,non_striker,bowler,runs_off_bat,extras,wides,noballs,byes,legbyes,penalty,wicket_type,player_dismissed,other_wicket_type,other_player_dismissed,match_id_innings,time_idx,total_runs_off_ball,cum_runs,is_wicket,cum_wickets,runs_prev_3,over_number,powerplay,death_overs,legal_delivery,legal_delivery_number,balls_remaining,city,toss_winner,toss_decision
0,211028,2005,2005-06-13,The Rose Bowl,1,0.1,England,Australia,ME Trescothick,GO Jones,B Lee,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,211028_1,1,0,0,0,0,0,1,1,0,1,1,119,Southampton,England,bat
1,211028,2005,2005-06-13,The Rose Bowl,1,0.2,England,Australia,ME Trescothick,GO Jones,B Lee,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,211028_1,2,1,1,0,0,0,1,1,0,1,2,118,Southampton,England,bat
2,211028,2005,2005-06-13,The Rose Bowl,1,0.3,England,Australia,GO Jones,ME Trescothick,B Lee,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,211028_1,3,0,1,0,0,0,1,1,0,1,3,117,Southampton,England,bat
3,211028,2005,2005-06-13,The Rose Bowl,1,0.4,England,Australia,GO Jones,ME Trescothick,B Lee,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,211028_1,4,0,1,0,0,0,1,1,0,1,4,116,Southampton,England,bat
4,211028,2005,2005-06-13,The Rose Bowl,1,0.5,England,Australia,GO Jones,ME Trescothick,B Lee,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,211028_1,5,0,1,0,0,1,1,1,0,1,5,115,Southampton,England,bat
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
122,211028,2005,2005-06-13,The Rose Bowl,1,19.4,England,Australia,VS Solanki,PD Collingwood,GD McGrath,0,0,NaN,NaN,NaN,NaN,NaN,caught,VS Solanki,NaN,NaN,211028_1,123,0,175,1,7,2,20,0,1,1,118,2,Southampton,England,bat
123,211028,2005,2005-06-13,The Rose Bowl,1,19.5,England,Australia,PD Collingwood,J Lewis,GD McGrath,4,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,211028_1,124,4,179,0,7,4,20,0,1,1,119,1,Southampton,England,bat
124,211028,2005,2005-06-13,The Rose Bowl,1,19.6,England,Australia,PD Collingwood,J Lewis,GD McGrath,0,0,NaN,NaN,NaN,NaN,NaN,caught,PD Collingwood,NaN,NaN,211028_1,125,0,179,1,8,2,20,0,1,1,120,0,Southampton,England,bat
125,211028,2005,2005-06-13,The Rose Bowl,2,0.1,Australia,England,AC Gilchrist,ML Hayden,D Gough,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,211028_2,1,0,0,0,0,0,1,1,0,1,1,119,Southampton,England,bat


Lets do some idiot checks!

In [ ]:
#number of unique match ids, there should be 26...
df_all_eng_aus['match_id'].nunique()

# lets go

26

In [ ]:
# balls per match
df_all_eng_aus['match_id'].value_counts()

#all look about right, the shorter games are due to match abadonment and domination

,count
match_id,
1317486,255
1317487,254
636164,251
1415717,251
566937,250
566938,249
249227,249
1198235,248
743975,246


In [ ]:
df_all_eng_aus[df_all_eng_aus['match_id'] == 350050].head()


,match_id,season,start_date,venue,innings,ball,batting_team,bowling_team,striker,non_striker,bowler,runs_off_bat,extras,wides,noballs,byes,legbyes,penalty,wicket_type,player_dismissed,other_wicket_type,other_player_dismissed,match_id_innings,time_idx,total_runs_off_ball,cum_runs,is_wicket,cum_wickets,runs_prev_3,over_number,powerplay,death_overs,city,toss_winner,toss_decision
676,350050,2009,2009-08-30,Old Trafford,1,0.1,Australia,England,SR Watson,DA Warner,JM Anderson,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,350050_1,1,0,0,0,0,0,1,1,0,Manchester,England,field
677,350050,2009,2009-08-30,Old Trafford,1,0.2,Australia,England,SR Watson,DA Warner,JM Anderson,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,350050_1,2,1,1,0,0,0,1,1,0,Manchester,England,field
678,350050,2009,2009-08-30,Old Trafford,1,0.3,Australia,England,DA Warner,SR Watson,JM Anderson,2,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,350050_1,3,2,3,0,0,0,1,1,0,Manchester,England,field
679,350050,2009,2009-08-30,Old Trafford,1,0.4,Australia,England,DA Warner,SR Watson,JM Anderson,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,350050_1,4,0,3,0,0,0,1,1,0,Manchester,England,field
680,350050,2009,2009-08-30,Old Trafford,1,0.5,Australia,England,DA Warner,SR Watson,JM Anderson,0,1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,350050_1,5,1,4,0,0,1,1,1,0,Manchester,England,field


In [ ]:
df_all_eng_aus[df_all_eng_aus['match_id_innings'] == '211028_1'][['time_idx', 'ball', 'legal_delivery', 'legal_delivery_number', 'balls_remaining']].tail(10)
# looks like balls_remaining works

,time_idx,ball,legal_delivery,legal_delivery_number,balls_remaining
115,116,18.3,1,111,9
116,117,18.4,1,112,8
117,118,18.5,1,113,7
118,119,18.6,1,114,6
119,120,19.1,1,115,5
120,121,19.2,1,116,4
121,122,19.3,1,117,3
122,123,19.4,1,118,2
123,124,19.5,1,119,1
124,125,19.6,1,120,0


### Target Variables
Add target variables to the process match function

In [ ]:
# function tp add all the features i have already created
def process_match(ball_df, info_dict):
    df = ball_df.copy()

    # match_id_innings
    df['match_id_innings'] = df['match_id'].astype(str) + "_" + df['innings'].astype(str)
    # time_idx
    df['time_idx'] = df.groupby('match_id_innings').cumcount() + 1
    # Total runs off ball
    df['total_runs_off_ball'] = df['runs_off_bat'] + df['extras']
    # Cumulative runs
    df['cum_runs'] = df.groupby('match_id_innings')['total_runs_off_ball'].cumsum()
    # wickets
    df['is_wicket'] = df['wicket_type'].notna().astype(int)
    df['cum_wickets'] = df.groupby('match_id_innings')['is_wicket'].cumsum()
    # previous 3 balls
    df['runs_prev_3'] = df.groupby('match_id_innings')['total_runs_off_ball'].shift(3).fillna(0).astype(int)
    # Over number
    df['over_number'] = df['ball'].apply(lambda x: int(x) + 1)
    # Powerplay and death overs
    df['powerplay'] = (df['over_number'] <= 6).astype(int)
    df['death_overs'] = (df['over_number'] >= 17).astype(int)

    # Static covariates from info file
    for key in ['city', 'toss_winner', 'toss_decision']:
        df[key] = info_dict.get(key, None)
    # Balls remaining
    #legal delivery column, cum count it then subtract from 120
    df['legal_delivery'] = ((df['wides'].fillna(0) == 0) & (df['noballs'].fillna(0) == 0)).astype(int)
    df['legal_delivery_number'] = df.groupby('match_id_innings')['legal_delivery'].cumsum()
    df['balls_remaining'] = 120 - df['legal_delivery_number']

    # add target variables
    # One-hot encoded runs
    for i in [0, 1, 2, 3, 4, 6]:
        df[f'rob_{i}'] = (df['runs_off_bat'] == i).astype(int)

    # Added this just so I have all target varibales together can delete is_wicket later
    df['wicket'] = df['is_wicket']

    # make extras binary not a value
    df['extra'] = (df['extras'] > 0).astype(int)

    # "Other" = not a given run, wicket, or extra. There is some wierd things that happen in cricketgames so I will have this as failsafe
    df['other'] = (
        (df[[f'rob_{i}' for i in [0,1,2,3,4,6]]].sum(axis=1) == 0) &
        (df['wicket'] == 0) &
        (df['extra'] == 0)
    ).astype(int)



    return df


Update df

In [ ]:
zip_path = "eng_aus_matches_only.zip"

all_eng_aus_matches = []

# Open the zip file
with zipfile.ZipFile(zip_path, "r") as zip_ref:
    for match_id in match_ids:
        ball_file = f"{match_id}.csv"
        info_file = f"{match_id}_info.csv"

        with zip_ref.open(ball_file) as f:
            ball_df = pd.read_csv(f)
            # look at the ball by ball file

        info_dict = {}
        with zip_ref.open(info_file) as f:
            for line in f:
                parts = line.decode('utf-8').strip().split(",")
                if len(parts) == 3 and parts[0] == "info":
                    key = parts[1].strip()
                    value = parts[2].strip()
                    info_dict[key] = value
                    # parse the info file for static covariates

        processed_df = process_match(ball_df, info_dict)
        all_eng_aus_matches.append(processed_df)

df_all_eng_aus = pd.concat(all_eng_aus_matches, ignore_index=True)

df_all_eng_aus.head()


,match_id,season,start_date,venue,innings,ball,batting_team,bowling_team,striker,non_striker,bowler,runs_off_bat,extras,wides,noballs,byes,legbyes,penalty,wicket_type,player_dismissed,other_wicket_type,other_player_dismissed,match_id_innings,time_idx,total_runs_off_ball,cum_runs,is_wicket,cum_wickets,runs_prev_3,over_number,powerplay,death_overs,city,toss_winner,toss_decision,legal_delivery,legal_delivery_number,balls_remaining,rob_0,rob_1,rob_2,rob_3,rob_4,rob_6,wicket,extra,other
0,211028,2005,2005-06-13,The Rose Bowl,1,0.1,England,Australia,ME Trescothick,GO Jones,B Lee,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,211028_1,1,0,0,0,0,0,1,1,0,Southampton,England,bat,1,1,119,1,0,0,0,0,0,0,0,0
1,211028,2005,2005-06-13,The Rose Bowl,1,0.2,England,Australia,ME Trescothick,GO Jones,B Lee,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,211028_1,2,1,1,0,0,0,1,1,0,Southampton,England,bat,1,2,118,0,1,0,0,0,0,0,0,0
2,211028,2005,2005-06-13,The Rose Bowl,1,0.3,England,Australia,GO Jones,ME Trescothick,B Lee,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,211028_1,3,0,1,0,0,0,1,1,0,Southampton,England,bat,1,3,117,1,0,0,0,0,0,0,0,0
3,211028,2005,2005-06-13,The Rose Bowl,1,0.4,England,Australia,GO Jones,ME Trescothick,B Lee,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,211028_1,4,0,1,0,0,0,1,1,0,Southampton,England,bat,1,4,116,1,0,0,0,0,0,0,0,0
4,211028,2005,2005-06-13,The Rose Bowl,1,0.5,England,Australia,GO Jones,ME Trescothick,B Lee,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,211028_1,5,0,1,0,0,1,1,1,0,Southampton,England,bat,1,5,115,1,0,0,0,0,0,0,0,0


In [ ]:
# look how many instances there are of each one
df_all_eng_aus[['rob_0', 'rob_1', 'rob_2', 'rob_3', 'rob_4', 'rob_6', 'wicket', 'extra', 'other']].sum()


,0
rob_0,2290
rob_1,2104
rob_2,501
rob_3,44
rob_4,702
rob_6,307
wicket,329
extra,315
other,1


In [ ]:
a = 2290+2104+501+44+702+307+329+315+1
print(a)

6593


In [ ]:
len(df_all_eng_aus)

5949

The no. of istances and lentgh of df are different because some balls are runs of bat and extras i.e hit a four off a no-ball. Need to work out what I am going to do here.

In [ ]:
df_all_eng_aus[df_all_eng_aus['other'] == 1]

# the one instance of other... travis head hit 5 off the bat!

# why is there 97 balls remaining when its delivery 3.6?
# must of been a wide/no-ball in the over... lets have a look

,match_id,season,start_date,venue,innings,ball,batting_team,bowling_team,striker,non_striker,bowler,runs_off_bat,extras,wides,noballs,byes,legbyes,penalty,wicket_type,player_dismissed,other_wicket_type,other_player_dismissed,match_id_innings,time_idx,total_runs_off_ball,cum_runs,is_wicket,cum_wickets,runs_prev_3,over_number,powerplay,death_overs,city,toss_winner,toss_decision,legal_delivery,legal_delivery_number,balls_remaining,rob_0,rob_1,rob_2,rob_3,rob_4,rob_6,wicket,extra,other
3574,1119542,2018,2018-06-27,Edgbaston,2,3.6,Australia,England,TM Head,AJ Finch,CJ Jordan,5,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1119542_2,25,5,38,0,2,1,4,1,0,Birmingham,Australia,field,1,23,97,0,0,0,0,0,0,0,0,1


In [ ]:
df_all_eng_aus[
    (df_all_eng_aus['match_id'] == 1119542) &
    (df_all_eng_aus['innings'] == 2) &
    (df_all_eng_aus['ball'].astype(str).str.startswith('3.'))
]
# yes ball 3 was a no ball

,match_id,season,start_date,venue,innings,ball,batting_team,bowling_team,striker,non_striker,bowler,runs_off_bat,extras,wides,noballs,byes,legbyes,penalty,wicket_type,player_dismissed,other_wicket_type,other_player_dismissed,match_id_innings,time_idx,total_runs_off_ball,cum_runs,is_wicket,cum_wickets,runs_prev_3,over_number,powerplay,death_overs,city,toss_winner,toss_decision,legal_delivery,legal_delivery_number,balls_remaining,rob_0,rob_1,rob_2,rob_3,rob_4,rob_6,wicket,extra,other
3569,1119542,2018,2018-06-27,Edgbaston,2,3.1,Australia,England,GJ Maxwell,AJ Finch,CJ Jordan,2,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1119542_2,20,2,26,0,1,1,4,1,0,Birmingham,Australia,field,1,19,101,0,0,1,0,0,0,0,0,0
3570,1119542,2018,2018-06-27,Edgbaston,2,3.2,Australia,England,GJ Maxwell,AJ Finch,CJ Jordan,2,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1119542_2,21,2,28,0,1,1,4,1,0,Birmingham,Australia,field,1,20,100,0,0,1,0,0,0,0,0,0
3571,1119542,2018,2018-06-27,Edgbaston,2,3.3,Australia,England,GJ Maxwell,AJ Finch,CJ Jordan,0,1,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1119542_2,22,1,29,0,1,4,4,1,0,Birmingham,Australia,field,0,20,100,1,0,0,0,0,0,0,1,0
3572,1119542,2018,2018-06-27,Edgbaston,2,3.4,Australia,England,GJ Maxwell,AJ Finch,CJ Jordan,4,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1119542_2,23,4,33,0,1,2,4,1,0,Birmingham,Australia,field,1,21,99,0,0,0,0,1,0,0,0,0
3573,1119542,2018,2018-06-27,Edgbaston,2,3.5,Australia,England,GJ Maxwell,AJ Finch,CJ Jordan,0,0,NaN,NaN,NaN,NaN,NaN,bowled,GJ Maxwell,NaN,NaN,1119542_2,24,0,33,1,2,2,4,1,0,Birmingham,Australia,field,1,22,98,1,0,0,0,0,0,1,0,0
3574,1119542,2018,2018-06-27,Edgbaston,2,3.6,Australia,England,TM Head,AJ Finch,CJ Jordan,5,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1119542_2,25,5,38,0,2,1,4,1,0,Birmingham,Australia,field,1,23,97,0,0,0,0,0,0,0,0,1
3575,1119542,2018,2018-06-27,Edgbaston,2,3.7,Australia,England,AJ Finch,TM Head,CJ Jordan,1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1119542_2,26,1,39,0,2,4,4,1,0,Birmingham,Australia,field,1,24,96,0,1,0,0,0,0,0,0,0


In [ ]:
df_all_eng_aus[df_all_eng_aus['other_player_dismissed'].notna()]
# no instances of two wickets on one ball
# looking at the rules it is not possible for this to happen

,match_id,season,start_date,venue,innings,ball,batting_team,bowling_team,striker,non_striker,bowler,runs_off_bat,extras,wides,noballs,byes,legbyes,penalty,wicket_type,player_dismissed,other_wicket_type,other_player_dismissed,match_id_innings,time_idx,total_runs_off_ball,cum_runs,is_wicket,cum_wickets,runs_prev_3,over_number,powerplay,death_overs,city,toss_winner,toss_decision,legal_delivery,legal_delivery_number,balls_remaining,rob_0,rob_1,rob_2,rob_3,rob_4,rob_6,wicket,extra,other


In [ ]:
df_all_eng_aus.shape

(5949, 47)

# **Setting up simple TFT**

I now need to define and split my varibales into 4 groups so that it is suitable for the DARTS documentation
1. Target
2. Past covariates
3. Future Covariates
4. Static covariates

Then we can convert to darts.timeseries objects

Each TimeSeries object is a sequence of data points (balls) within that specific innings

In [ ]:
!pip install darts


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.0/56.0 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.3/46.3 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.6/200.6 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 825.4/825.4 kB 47.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.7/37.7 MB 49.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 340.0/340.0 kB 25.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 55.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 34.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8

In [ ]:
from darts import TimeSeries
from darts.models import TFTModel
from darts.utils.timeseries_generation import datetime_attribute_timeseries
from darts.dataprocessing.transformers import Scaler
from darts.utils.likelihood_models.torch import QuantileRegression
from sklearn.preprocessing import LabelEncoder


In [ ]:
df_all_eng_aus.head(0)

,match_id,season,start_date,venue,innings,ball,batting_team,bowling_team,striker,non_striker,bowler,runs_off_bat,extras,wides,noballs,byes,legbyes,penalty,wicket_type,player_dismissed,other_wicket_type,other_player_dismissed,match_id_innings,time_idx,total_runs_off_ball,cum_runs,is_wicket,cum_wickets,runs_prev_3,over_number,powerplay,death_overs,city,toss_winner,toss_decision,legal_delivery,legal_delivery_number,balls_remaining,rob_0,rob_1,rob_2,rob_3,rob_4,rob_6,wicket,extra,other


In [ ]:
# specify which columns to extract from the master df
target_cols = ['rob_0', 'rob_1', 'rob_2', 'rob_3', 'rob_4', 'rob_6', 'wicket', 'extra', 'other']
past_cov_cols = ['time_idx', 'over_number', 'cum_runs', 'cum_wickets', 'runs_prev_3', 'legal_delivery', 'balls_remaining']
future_cov_cols = ['powerplay', 'death_overs', 'over_number']
static_cov_cols = ['venue', 'toss_winner', 'toss_decision']

#still need to encode static variables

Label encode statics as Darts expects all covariates to be numeric

In [ ]:
# Label encode static covariates, each venue will have a unique number same with toss winner, toss decision
for col in ['venue', 'toss_winner', 'toss_decision']:
    df_all_eng_aus[col] = LabelEncoder().fit_transform(df_all_eng_aus[col].astype(str))



### Convert innings to darts time series

In [ ]:

# Convert each innings into TimeSeries format
def create_series_grouped(df):
    target_series = []
    past_cov_series = []
    future_cov_series = []
    static_covs = []

    for _, group in df.groupby("match_id_innings"):
        group = group.sort_values("time_idx")
# group each innings and then ensure order by time_idx

        ts_target = TimeSeries.from_dataframe(group, value_cols=target_cols, time_col="time_idx")
        ts_past = TimeSeries.from_dataframe(group, value_cols=past_cov_cols, time_col="time_idx")
        ts_future = TimeSeries.from_dataframe(group, value_cols=future_cov_cols, time_col="time_idx")
        ts_static = group[static_cov_cols].iloc[0].to_numpy(dtype=np.float32) # convert to float as this is what darts expects
# create Timeseries for each of our targets and covariates

        target_series.append(ts_target)
        past_cov_series.append(ts_past)
        future_cov_series.append(ts_future)
        static_covs.append(ts_static)
# append will add innings by innings

    return target_series, past_cov_series, future_cov_series, static_covs

# Run it
target_series, past_covs, future_covs, static_covs = create_series_grouped(df_all_eng_aus)


In [ ]:
df_all_eng_aus['match_id'].nunique()


26

In [ ]:
df_all_eng_aus['match_id_innings'].nunique()

52

In [ ]:
len(target_series)

52

In [ ]:
len(past_covs)

52

In [ ]:
# check what the first time series looks like

print("Target Series[0] shape:", target_series[0].values().shape)
print("Target Series[0] sample values:\n", target_series[0].values()[:5])


print("\nPast Covariates[0] shape:", past_covs[0].values().shape)
print("Past Covariates[0] sample values:\n", past_covs[0].values()[:5])


print("\nFuture Covariates[0] shape:", future_covs[0].values().shape)
print("Future Covariates[0] sample values:\n", future_covs[0].values()[:5])


print("\nStatic Covariates[0] shape:", static_covs[0].shape)
print("Static Covariates[0] values:", static_covs[0])

# the shape being 124 is the number of balls

Target Series[0] shape: (124, 9)
Target Series[0] sample values:
 [[0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0.]]

Past Covariates[0] shape: (124, 7)
Past Covariates[0] sample values:
 [[  1.   1.   2.   0.   0.   1. 119.]
 [  2.   1.   2.   0.   0.   1. 118.]
 [  3.   1.   3.   0.   0.   0. 118.]
 [  4.   1.   7.   0.   2.   1. 117.]
 [  5.   1.   9.   0.   0.   1. 116.]]

Future Covariates[0] shape: (124, 3)
Future Covariates[0] sample values:
 [[1. 0. 1.]
 [1. 0. 1.]
 [1. 0. 1.]
 [1. 0. 1.]
 [1. 0. 1.]]

Static Covariates[0] shape: (3,)
Static Covariates[0] values: [1. 0. 1.]


### Train Val Test split

In [ ]:
from sklearn.model_selection import train_test_split

train_targets, test_targets, train_past, test_past, train_future, test_future, train_static, test_static = train_test_split(
    target_series, past_covs, future_covs, static_covs, test_size=0.15, random_state=42
)
# getting the test set

train_targets, val_targets, train_past, val_past, train_future, val_future, train_static, val_static = train_test_split(
    train_targets, train_past, train_future, train_static, test_size=0.18, random_state=42
)
# split remaining samples into train and val sets

# check how many samples in each set
print(f"Train samples: {len(train_targets)}")
print(f"Validation samples: {len(val_targets)}")
print(f"Test samples: {len(test_targets)}")


Train samples: 36
Validation samples: 8
Test samples: 8


Have had to add this filter because some innings where to short

**Need to deal with this - not to self**

In [ ]:
# Get the TimeSeries at index 18
ts_example = train_targets[18]

# index 18 was to short
print(f"Length of innings at index 18: {len(ts_example)}")
print("Time index:", ts_example.time_index)
print("Values:")
print(ts_example.values())


Length of innings at index 18: 10
Time index: RangeIndex(start=1, stop=11, step=1, name='time_idx')
Values:
[[1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 1. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 1. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 1. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 1. 0.]
 [1. 0. 0. 0. 0. 0. 1. 0. 0.]]


In [ ]:
def filter_short_series(targets, past, future, static, min_len=31):
    filtered = [
        (t, p, f, s)
        for t, p, f, s in zip(targets, past, future, static)
        if len(t) >= min_len
    ]
    return zip(*filtered) if filtered else ([], [], [], [])

# Apply to each dataset
train_targets, train_past, train_future, train_static = filter_short_series(
    train_targets, train_past, train_future, train_static
)
val_targets, val_past, val_future, val_static = filter_short_series(
    val_targets, val_past, val_future, val_static
)
test_targets, test_past, test_future, test_static = filter_short_series(
    test_targets, test_past, test_future, test_static
)


### Time to scale
Only need to scale past and future covariates

In [ ]:
# scale past covariates
scaler_past = Scaler()
train_past_scaled = scaler_past.fit_transform(train_past)
# fit and scale train set
val_past_scaled = scaler_past.transform(val_past)
test_past_scaled = scaler_past.transform(test_past)
# dont fit scaler just apply

# Scale future covariates
scaler_future = Scaler()
# new scaler
train_future_scaled = scaler_future.fit_transform(train_future)
# fit and scale on train set
val_future_scaled = scaler_future.transform(val_future)
test_future_scaled = scaler_future.transform(test_future)
# just scale train and val set


train_targets = [ts.with_static_covariates(pd.Series(sc, index=static_cov_cols)) for ts, sc in zip(train_targets, train_static)]
val_targets = [ts.with_static_covariates(pd.Series(sc, index=static_cov_cols)) for ts, sc in zip(val_targets, val_static)]
test_targets = [ts.with_static_covariates(pd.Series(sc, index=static_cov_cols)) for ts, sc in zip(test_targets, test_static)]

#insert the static covariates back in. When using .with_static_covariates needs to be a pd series or pd df
#loops once per innings and inserts my static covariates

In [ ]:
print(len(train_past))
print(len(val_past))
print(len(test_past))

35
8
7


### Model Setup

In [ ]:
quantiles = [0.1, 0.5, 0.9]  # Adjust later if needed

model = TFTModel(
    input_chunk_length=30,       # look sback 30 balls, can chnage this
    output_chunk_length=1,       # predict 1 ball
    hidden_size=32,              # lightweight start, can increase
    lstm_layers=1,               # single layer to start as not much data
    num_attention_heads=4,       # darts reccomended default
    dropout=0.1,                 # prevent overfitting
    batch_size=32,               # efficeint to start, could increase if switch compute
    n_epochs=50,                 # default is 100, short to start then increase
    add_relative_index=True,     # adds positional value to future covariates
    likelihood=QuantileRegression(quantiles=quantiles),   #default
    random_state=42,
    force_reset=True,            #training resets each time
)


Fit model

In [ ]:
model.fit(
    series=train_targets,
    past_covariates=train_past_scaled,
    future_covariates=train_future_scaled,
    verbose=True
)

# not how long this model takes to fit even with the current set up
# it took 17 mins

INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.callbacks.model_summary:
   | Name                              | Type                             | Params | Mode 
------------------------------------------------------------------------------------------------
0  | train_metrics                     | MetricCollection                 | 0      | train
1  | val_metrics                       | MetricCollection                 | 0      | train
2  | input_embeddings                  | _MultiEmbedding                  | 0      | train
3  | static_covariates_vsn             | _VariableSelectionNetwork        | 2.8 K  | train
4  | encoder_vsn                       | _VariableSelectionNetwork        | 23.1 K | train
5  | decoder_vsn                       | _VariableSelectionNetwork   

Training: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=50` reached.


TFTModel(output_chunk_shift=0, hidden_size=32, lstm_layers=1, num_attention_heads=4, full_attention=False, feed_forward=GatedResidualNetwork, dropout=0.1, hidden_continuous_size=8, categorical_embedding_sizes=None, add_relative_index=True, loss_fn=None, likelihood=QuantileRegression(quantiles=[0.1, 0.5, 0.9], prior_strength=1.0), norm_type=LayerNorm, use_static_covariates=True, input_chunk_length=30, output_chunk_length=1, batch_size=32, n_epochs=50, random_state=42, force_reset=True)

Make predictions on val

In [ ]:
# Trim only target and past covariates
val_targets_trimmed = [ts[:-1] for ts in val_targets]
val_past_trimmed = [ts[:-1] for ts in val_past_scaled]
# DO NOT TRIM future covariates
val_future_trimmed = val_future_scaled

# Predict 1 step ahead
val_predictions = model.predict(
    n=1,
    series=val_targets_trimmed,
    past_covariates=val_past_trimmed,
    future_covariates=val_future_trimmed,
    num_samples=100
)


INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


Predicting: |          | 0/? [00:00<?, ?it/s]

In [ ]:
val_predictions = model.predict(
    n=1,  # predict one step ahead (since your output_chunk_length=1)
    series=val_targets,
    past_covariates=val_past_scaled,
    future_covariates=val_future_scaled,
    num_samples=100
)


ERROR:darts.utils.data.torch_datasets.dataset:ValueError: For the given forecasting horizon `n=1`, the provided `future_covariates` at series sequence index `0` do not extend far enough into the future. As `n <= output_chunk_length` the `future_covariates` must end at or after time step `122`, whereas now the end is at time step `121`.


ValueError: For the given forecasting horizon `n=1`, the provided `future_covariates` at series sequence index `0` do not extend far enough into the future. As `n <= output_chunk_length` the `future_covariates` must end at or after time step `122`, whereas now the end is at time step `121`.

In [ ]:
val_predictions[0].plot(label="Predicted")
val_targets[0].plot(label="Actual")


Ananlysis

In [ ]:
# Actual class probability
# Combine all actual target series from the validation set
actual_vals = pd.concat([ts.pd_dataframe() for ts in val_targets])

# Compute mean for each class
actual_class_probs = actual_vals.mean()
print("Actual Class Probabilities:")
print(actual_class_probs)


In [ ]:
#Predicted class probability
# Convert all probabilistic predictions to means
predicted_vals = pd.concat([ts.mean(axis=0).pd_dataframe() for ts in val_predictions])

# Compute mean predicted probability for each class
predicted_class_probs = predicted_vals.mean()
print("Predicted Class Probabilities:")
print(predicted_class_probs)


In [ ]:
#Compare
comparison_df = pd.DataFrame({
    "Actual": actual_class_probs,
    "Predicted": predicted_class_probs
})

print(comparison_df)


In [ ]:
import matplotlib.pyplot as plt

# Combine into a single df
comparison_df = pd.DataFrame({
    "Actual": actual_class_probs,
    "Predicted": predicted_class_probs
})

# Plot
comparison_df.plot(kind='bar', figsize=(10, 6))
plt.title("Predicted vs Actual Class Probabilities (Validation Set)")
plt.ylabel("Probability")
plt.xlabel("Outcome Class")
plt.xticks(rotation=45)
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()
